## Deliverable 2. Create a Customer Travel Destinations Map.

In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Fortuna,US,40.5982,-124.1573,63.99,93,94,5.88,overcast clouds
1,1,Atuona,PF,-9.8000,-139.0333,76.87,82,37,27.69,light rain
2,2,Constitucion,CL,-35.3333,-72.4167,47.70,85,100,6.06,overcast clouds
3,3,Qaanaaq,GL,77.4840,-69.3632,36.64,79,80,2.04,broken clouds
4,4,Albany,US,42.6001,-73.9662,70.57,79,99,1.77,overcast clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()


City_ID                256
City                   256
Country                256
Lat                    256
Lng                    256
Max Temp               256
Humidity               256
Cloudiness             256
Wind Speed             256
Current Description    256
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()


In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Atuona,PF,76.87,light rain,-9.8000,-139.0333,
6,Cochrane,CA,81.64,clear sky,51.1834,-114.4687,
12,Georgetown,MY,78.73,few clouds,5.4112,100.3354,
13,Baghdad,IQ,89.51,clear sky,33.3406,44.4009,
17,Butaritari,KI,81.70,overcast clouds,3.0707,172.7902,
18,Havelock,US,87.94,overcast clouds,34.8791,-76.9013,
23,Buenos Aires,AR,76.98,clear sky,-34.6132,-58.3772,
25,Wiarton,CA,76.03,clear sky,44.7407,-81.1332,
26,Port-Gentil,GA,78.76,broken clouds,-0.7193,8.7815,
29,Kahului,US,84.22,few clouds,20.8947,-156.4700,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print(str(index) + ": Hotel Name in <" + hotel_df["City"] + "> not found... skipping.")
        pass
        

1      43: Hotel Name in <Atuona> not found... skipping.
6      43: Hotel Name in <Cochrane> not found... skip...
12     43: Hotel Name in <Georgetown> not found... sk...
13     43: Hotel Name in <Baghdad> not found... skipp...
17     43: Hotel Name in <Butaritari> not found... sk...
                             ...                        
705    43: Hotel Name in <Karpathos> not found... ski...
710    43: Hotel Name in <Yaring> not found... skipping.
712    43: Hotel Name in <Ballesteros> not found... s...
714    43: Hotel Name in <Thinadhoo> not found... ski...
715    43: Hotel Name in <Mwene-Ditu> not found... sk...
Name: City, Length: 256, dtype: object
1      136: Hotel Name in <Atuona> not found... skipp...
6      136: Hotel Name in <Cochrane> not found... ski...
12     136: Hotel Name in <Georgetown> not found... s...
13     136: Hotel Name in <Baghdad> not found... skip...
17     136: Hotel Name in <Butaritari> not found... s...
                             ...                 

1      714: Hotel Name in <Atuona> not found... skipp...
6      714: Hotel Name in <Cochrane> not found... ski...
12     714: Hotel Name in <Georgetown> not found... s...
13     714: Hotel Name in <Baghdad> not found... skip...
17     714: Hotel Name in <Butaritari> not found... s...
                             ...                        
705    714: Hotel Name in <Karpathos> not found... sk...
710    714: Hotel Name in <Yaring> not found... skipp...
712    714: Hotel Name in <Ballesteros> not found... ...
714    714: Hotel Name in <Thinadhoo> not found... sk...
715    714: Hotel Name in <Mwene-Ditu> not found... s...
Name: City, Length: 256, dtype: object


In [10]:
# 7. Drop the rows where there is no Hotel Name

clean_hotel_df = hotel_df[(hotel_df["Hotel Name"] != "")]

# Only keep the first 180 rows
clean_hotel_df = clean_hotel_df.head(180)

# Confirm the data looks good
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Atuona,PF,76.87,light rain,-9.8000,-139.0333,Hôtel Hiva Oa Hanakee Lodge
6,Cochrane,CA,81.64,clear sky,51.1834,-114.4687,Days Inn & Suites by Wyndham Cochrane
12,Georgetown,MY,78.73,few clouds,5.4112,100.3354,Cititel Penang
13,Baghdad,IQ,89.51,clear sky,33.3406,44.4009,Baghdad Hotel
17,Butaritari,KI,81.70,overcast clouds,3.0707,172.7902,Isles Sunset Lodge
18,Havelock,US,87.94,overcast clouds,34.8791,-76.9013,Sherwood Motel
23,Buenos Aires,AR,76.98,clear sky,-34.6132,-58.3772,Hotel NH Buenos Aires City
25,Wiarton,CA,76.03,clear sky,44.7407,-81.1332,Waterview On The Bay
26,Port-Gentil,GA,78.76,broken clouds,-0.7193,8.7815,Fengshui Residence
29,Kahului,US,84.22,few clouds,20.8947,-156.4700,Maui Seaside Hotel


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]


In [13]:
# 11a. Add a marker layer for each city to the map. 
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))